In [2]:
import numpy as np
import tensorflow as tf

In [3]:

def conv_net(tensor, num_outputs, ksize_r, ksize_c, stride_r, stride_c):
    weights = tf.Variable(
        tf.random_normal([
            ksize_r, 
            ksize_c,
            3,
            num_outputs]), tf.float32)
    biases = tf.Variable(tf.zeros(num_outputs))
    tensor = tf.nn.conv2d(tensor, weights, [1, stride_r, stride_c,1], padding='SAME')
    tensor = tf.nn.max_pool(tensor, [1, 3, 3, 1], [1, 3, 3, 1], padding='SAME')
    tensor = tf.nn.bias_add(tensor, biases)
    tensor = tf.nn.relu(tensor)
    return tensor
    

In [4]:
def full_connected(tensor, num_in, num_out):
    weights = tf.Variable(tf.random_normal([num_in, num_out]))
    biases = tf.Variable(tf.zeros([num_out]))
    tensor = tf.matmul(tensor, num_in)
    tensor = tf.add(tensor, biases)
    tensor = tf.nn.relu(tensor)
    return tensor

In [5]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 28, 28,3], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')


model = conv_net(x, 10, 3, 3, 1, 1)


# Cost & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(cost)

# accuracy
correct_pred = tf.equal(tf.arg_max(model, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


In [ ]:

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        sess.run(tf.global_variables_initializer())
        train_image = np.zeros([2, 28, 28,3])
        train_labels = np.zeros([2, 10])
        sess.run(accuracy, feed_dict={x:train_image, y:train_labels})
        writer = tf.summary.FileWriter("tboard_data")
        writer.add_graph(sess.graph)
        print("Done")
